<a href="https://colab.research.google.com/github/santiago2588/Pump_failure_training/blob/main/soluciones/05_xgboost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

In [2]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight

In [3]:
# Download and run the utils.py file from your GitHub repository
!wget https://raw.githubusercontent.com/santiago2588/pump_failure_training/main/utils.py
%run utils.py

--2025-07-15 10:30:30--  https://raw.githubusercontent.com/santiago2588/pump_failure_training/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2407 (2.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   2.35K  --.-KB/s    in 0s      

2025-07-15 10:30:30 (29.9 MB/s) - ‘utils.py’ saved [2407/2407]



In [4]:
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline

In [5]:
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import f1_score

In [6]:
%pip install optuna -q
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.5 MB/s eta 0:00:00


In [7]:
!wget https://raw.githubusercontent.com/santiago2588/pump_failure_training/main/data/transformed_data.csv -O transformed_data.csv

# Read the CSV file into a DataFrame
df = pd.read_csv("transformed_data.csv")

# Display the first few rows to confirm it loaded correctly
df.head()

--2025-07-15 10:30:46--  https://raw.githubusercontent.com/santiago2588/pump_failure_training/main/data/transformed_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1210184 (1.2M) [text/plain]
Saving to: ‘transformed_data.csv’

transformed_data.cs 100%[===================>]   1.15M  --.-KB/s    in 0.06s   

2025-07-15 10:30:47 (19.1 MB/s) - ‘transformed_data.csv’ saved [1210184/1210184]



,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Type_High,Type_Low,Type_Medium,Failure_type
0,-0.951417,-0.946356,0.067484,0.283054,-1.695647,0.0,0.0,1.0,No Failure
1,-0.901428,-0.878954,-0.729604,0.634238,-1.648511,0.0,1.0,0.0,No Failure
2,-0.951417,-1.013759,-0.227940,0.945286,-1.617087,0.0,1.0,0.0,No Failure
3,-0.901428,-0.946356,-0.590253,-0.048061,-1.585664,0.0,1.0,0.0,No Failure
4,-0.901428,-0.878954,-0.729604,0.002108,-1.554240,0.0,1.0,0.0,No Failure


In [8]:
# Define features (X) and target variable (y)
X = df.drop('Failure_type', axis=1)  # Features (all columns except 'Failure_type')
y = df['Failure_type']  # Target variable

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (7978, 8)
Testing data shape: (1995, 8)


In [10]:
# Encode target labels as XGB requires numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [11]:
# Creating pipeline with xgboost classifier and smote
xg_boost_smote = Pipeline(steps=[
    ('smote', SMOTE(random_state=2023)),
    ('model', XGBClassifier(random_state=2023))
])

# Fit pipeline
xg_boost_smote.fit(X_train, y_train_encoded)

# Generate Predictions using the correctly fitted pipeline
y_pred = xg_boost_smote.predict(X_test)

# Decode predictions back to original labels (optional)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluate Metrics
metrics = get_metrics(y_test_encoded, y_pred)

# View Results
metrics

{'Accuracy': 0.9754385964912281,
 'Balanced Accuracy': np.float64(0.8494708947179304),
 'Macro Recall': 0.9754385964912281,
 'Macro Precision': 0.989242283791671,
 'Macro F1': 0.9815870281839686,
 'F1 Scores per Class': array([0.89795918, 0.98748044, 0.69565217, 0.85714286, 0.11428571])}

In [12]:
# Creating model with xgboost classifier and balanced class weights

# Compute sample weights for class imbalance
weights = compute_sample_weight(class_weight='balanced', y=y_train_encoded)

# Initialize XGBoost classifier
xgb_model = XGBClassifier(random_state=2023)

# Fit the model with sample weights
xgb_model.fit(X_train, y_train_encoded, sample_weight=weights)

# Generate predictions
y_pred = xgb_model.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test_encoded, y_pred)

# View results
metrics

{'Accuracy': 0.9829573934837093,
 'Balanced Accuracy': np.float64(0.8574281351060007),
 'Macro Recall': 0.9829573934837093,
 'Macro Precision': 0.9890159415390738,
 'Macro F1': 0.9854568402093054,
 'F1 Scores per Class': array([0.88      , 0.9914308 , 0.66666667, 0.85106383, 0.28571429])}

In [13]:
# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),  # Number of trees
        'max_depth': trial.suggest_int('max_depth', 3, 15),  # Maximum depth of trees
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),  # Learning rate
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # Subsample ratio
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # Feature subsampling
        'gamma': trial.suggest_float('gamma', 0, 5),  # Minimum loss reduction
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),  # L2 regularization
    }

    # Initialize the XGBoost classifier with the trial's parameters
    model = XGBClassifier(random_state=2023, **params)


    # Compute sample weights for class imbalance
    weights = compute_sample_weight(class_weight='balanced', y=y_train_encoded)

    # Fit the model with sample weights
    model.fit(X_train, y_train_encoded, sample_weight=weights)

    # Generate predictions
    y_pred = model.predict(X_test)

    # Evaluate the model using F1 score (or any other metric)
    f1 = f1_score(y_test_encoded, y_pred, average='weighted')  # Weighted F1 score for imbalanced data

    return f1  # Optuna will maximize this score

In [14]:
# Create a study object
study = optuna.create_study(direction='maximize')  # We want to maximize F1-score

# Optimize the study
study.optimize(objective, n_trials=20)  # Run 20 trials (you can increase this for better results)

# Print the best hyperparameters
print("Best hyperparameters:", study.best_params)
print("Best F1-score:", study.best_value)

[I 2025-07-15 10:31:13,109] A new study created in memory with name: no-name-a18c57d0-a144-4247-b3a8-04fe0cc27ad3
[I 2025-07-15 10:31:19,469] Trial 0 finished with value: 0.9252415272770406 and parameters: {'n_estimators': 454, 'max_depth': 15, 'learning_rate': 0.060054944754584276, 'subsample': 0.5221243093662037, 'colsample_bytree': 0.5244060978618859, 'gamma': 3.1272390888691204, 'reg_alpha': 6.451343142387892, 'reg_lambda': 9.304181940822174}. Best is trial 0 with value: 0.9252415272770406.
[I 2025-07-15 10:31:20,484] Trial 1 finished with value: 0.9231419326341613 and parameters: {'n_estimators': 170, 'max_depth': 15, 'learning_rate': 0.013956821547500387, 'subsample': 0.5219034305111918, 'colsample_bytree': 0.8281229976071238, 'gamma': 4.053026095131398, 'reg_alpha': 4.531986527402459, 'reg_lambda': 1.1873117341452577}. Best is trial 0 with value: 0.9252415272770406.
[I 2025-07-15 10:31:20,914] Trial 2 finished with value: 0.9217633127106811 and parameters: {'n_estimators': 97, '

Best hyperparameters: {'n_estimators': 215, 'max_depth': 7, 'learning_rate': 0.095003901548279, 'subsample': 0.9495350753408955, 'colsample_bytree': 0.9165351345041708, 'gamma': 0.012482231800288032, 'reg_alpha': 8.320461091730966, 'reg_lambda': 5.292747815379002}
Best F1-score: 0.946563399570467


In [15]:
# Extract the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
final_model = XGBClassifier(random_state=2023, **best_params)

# Compute sample weights again
weights = compute_sample_weight(class_weight='balanced', y=y_train_encoded)

# Fit the final model
final_model.fit(X_train, y_train_encoded, sample_weight=weights)

# Generate predictions
y_pred = final_model.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test_encoded, y_pred)

# Remove 'F1 Scores per Class' from metrics
if 'F1 Scores per Class' in metrics:
    del metrics['F1 Scores per Class']

# View results
metrics

{'Accuracy': 0.9218045112781955,
 'Balanced Accuracy': np.float64(0.89549856270212),
 'Macro Recall': 0.9218045112781955,
 'Macro Precision': 0.9813844241033967,
 'Macro F1': 0.946563399570467}